<a href="https://colab.research.google.com/github/VitalyGladyshev/gb_nlp/blob/main/HW_05_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ДЗ 05 NLP

Берем отызывы за лето (из архива с материалами или предыдущего занятия)

In [1]:
import numpy as np
import pandas as pd

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
path_otzv = "/content/gdrive/My Drive/Colab Notebooks/NLP/отзывы за лето.xls"

In [4]:
data = pd.read_excel(path_otzv)

In [5]:
data.tail(3)

,Rating,Content,Date
20656,4,Доволен,2017-06-01
20657,1,"Песопаснасть, рут ни нужын",2017-06-01
20658,5,Сбербанк бомбовая компания на сегодняшний день...,2017-06-01


In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20659 entries, 0 to 20658
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Rating   20659 non-null  int64 
 1   Content  20656 non-null  object
 2   Date     20659 non-null  object
dtypes: int64(1), object(2)
memory usage: 484.3+ KB


In [19]:
from datetime import datetime

In [20]:
type(data["Date"].values[0])

str

In [36]:
datetime_object = datetime.strptime(data["Date"].values[0], '%Y-%m-%d').date()
datetime_object

datetime.date(2017, 8, 14)

In [34]:
data["Year"] = [datetime.strptime(dt, '%Y-%m-%d').year for dt in data["Date"].values]
data["Month"] = [datetime.strptime(dt, '%Y-%m-%d').month for dt in data["Date"].values]
data["Day"] = [datetime.strptime(dt, '%Y-%m-%d').day for dt in data["Date"].values]
data.head()

,Rating,Content,Date,Year,Month,Day
0,5,It just works!,2017-08-14,2017,8,14
1,4,В целом удобноное приложение...из минусов хотя...,2017-08-14,2017,8,14
2,5,Отлично все,2017-08-14,2017,8,14
3,5,Стал зависать на 1% работы антивируса. Дальше ...,2017-08-14,2017,8,14
4,5,"Очень удобно, работает быстро.",2017-08-14,2017,8,14


## Задание 1

1. Учим conv сеть для классификации - выбить auc выше 0.95

In [38]:
pip install stop_words

  Created wheel for stop-words: filename=stop_words-2018.7.23-cp36-none-any.whl size=32916 sha256=2499d3e4f69a90e2ab0e54cc95240b711816d477eaef720f12d0ad9afe57aaee
  Stored in directory: /root/.cache/pip/wheels/75/37/6a/2b295e03bd07290f0da95c3adb9a74ba95fbc333aa8b0c7c78
Successfully built stop-words


In [40]:
pip install pymorphy2

     |████████████████████████████████| 61kB 1.0MB/s 
     |████████████████████████████████| 8.2MB 500kB/s 


In [41]:
from string import punctuation
from stop_words import get_stop_words
from pymorphy2 import MorphAnalyzer
import re

In [42]:
sw = set(get_stop_words("ru"))
exclude = set(punctuation)
morpher = MorphAnalyzer()

def preprocess_text(txt):
    txt = str(txt)
    txt = "".join(c for c in txt if c not in exclude)
    txt = txt.lower()
    txt = re.sub("\sне", "не", txt)
    txt = [morpher.parse(word)[0].normal_form for word in txt.split() if word not in sw]
    return " ".join(txt)

In [43]:
data['Content_prep'] = data['Content'].apply(preprocess_text)
data.tail()

,Rating,Content,Date,Year,Month,Day,Content_prep
20654,1,"Ну и шляпа,с роот правами бесполезная прога,ра...",2017-06-01,2017,6,1,шляпас роот право бесполезный прогаразрабыв ох...
20655,5,Ок,2017-06-01,2017,6,1,около
20656,4,Доволен,2017-06-01,2017,6,1,довольный
20657,1,"Песопаснасть, рут ни нужын",2017-06-01,2017,6,1,песопаснастя рута нужын
20658,5,Сбербанк бомбовая компания на сегодняшний день...,2017-06-01,2017,6,1,сбербанк бомбовый компания сегодняшний лидер б...


In [46]:
train_corpus = " ".join(data['Content_prep'])

In [56]:
import nltk
from nltk.tokenize import word_tokenize
nltk.download("punkt")

tokens_tot = word_tokenize(train_corpus)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [57]:
tokens_filtered = [word for word in tokens_tot if word.isalnum()]

In [68]:
from nltk.probability import FreqDist
dist = FreqDist(tokens_filtered)

In [65]:
vocabulary = {v: i  for i, v in enumerate(dist) if dist[v]>1}

In [70]:
def text_to_sequence(text, maxlen):
    result = []
    tokens = word_tokenize(text.lower())
    tokens_filtered = [word for word in tokens if word.isalnum()]
    for word in tokens_filtered:
        if word in vocabulary:
            result.append(vocabulary[word])
    padding = [0]*(maxlen-len(result))
    return padding + result[-maxlen:]

In [72]:
print(text_to_sequence(data['Content_prep'].values[5737], 20))

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3304, 131, 304, 3184, 5156]


In [73]:
data_embeded = np.asarray([text_to_sequence(text, 20) for text in data['Content_prep']], dtype=np.int32)

In [74]:
data_embeded

array([[   0,    0,    0, ...,    0,    0,    2],
       [   0,    0,    0, ...,   13,   14,   15],
       [   0,    0,    0, ...,    0,    0,   16],
       ...,
       [   0,    0,    0, ...,    0,    0,  174],
       [   0,    0,    0, ...,    0,    0,  425],
       [   0,    0,    0, ..., 6321,  327, 3714]], dtype=int32)

In [77]:
from sklearn.model_selection import train_test_split

In [78]:
y = data['Rating'].values
train_x, valid_x, train_y, valid_y = train_test_split(data_embeded, y, stratify=y, random_state=43)

In [81]:
train_x.shape

(15494, 20)

In [83]:
tr_data = train_x.reshape(train_x.shape[0], train_x.shape[1], 1)
vd_data = valid_x.reshape(valid_x.shape[0], valid_x.shape[1], 1)
tr_data.shape

(15494, 20, 1)

In [99]:
def to_one_hot(Y):
    n_col = np.amax(Y)
    binarized = np.zeros((len(Y), n_col))
    for i in range(len(Y)):
        binarized[i, Y[i]-1] = 1.
    return binarized

In [101]:
train_y_o = to_one_hot(train_y)
valid_y_o = to_one_hot(valid_y)
train_y_o

array([[0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.],
       [0., 1., 0., 0., 0.],
       ...,
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.]])

### Строим свёрточную нейронную сеть

In [90]:
import tensorflow as tf

# !pip install -q tensorflow-hub
# !pip install -q tensorflow-datasets
import tensorflow_hub as hub
# import tensorflow_datasets as tfds

import keras

from keras.models import Sequential
from keras.layers import Dense, Conv1D, MaxPooling1D, Dropout, Flatten
from keras.utils import to_categorical

from typing import List, Optional, Dict, Tuple

In [91]:
def calculate_confidence_interval(scores: list, conf_interval: float = 0.95) -> Tuple[float]:
    """
    Вычисление доверительного интервала.

    Parameters
    ----------
    scores: List[float / int]
        Список с оценками изучаемой величины.

    conf_interval: float, optional, default = 0.95
        Уровень доверия для построения интервала.
        Опциональный параметр, по умолчанию, равен 0.95.

    Returns
    -------
    conf_interval: Tuple[float]
        Кортеж с границами доверительного интервала.

    """
    left_bound = np.percentile(
        scores, ((1 - conf_interval) / 2) * 100
    )
    right_bound = np.percentile(
        scores, (conf_interval + ((1 - conf_interval) / 2)) * 100
    )

    return left_bound, right_bound

In [181]:
def net_gen(lay_cnt=1, filters=32, neurons=32):
    # конфигурирование слоев нейросети
    model = Sequential()

    # слои нейросети отвественные за свертку и max-pooling
    model.add(Conv1D(filters, 3, padding='same', activation='sigmoid', input_shape=(tr_data.shape[1], 1)))   # relu
    # model.add(Conv1D(filters, 3, activation='relu'))
    # model.add(MaxPooling1D(pool_size=2))
    model.add(Dropout(0.25))
    if lay_cnt == 2:
        model.add(Conv1D(filters, 3, padding='same', activation='sigmoid')) # relu
        # model.add(MaxPooling1D(pool_size=2))
        model.add(Dropout(0.25))

    # полносвязные слои нейронной сети
    model.add(Flatten())
    model.add(Dense(neurons, activation='sigmoid'))   # relu
    # model.add(Dense(neurons, activation='relu'))))
    model.add(Dropout(0.5))
    model.add(Dense(5, activation='softmax')) # sigmoid'))

    # компиляция модели
    model.compile(optimizer=tf.keras.optimizers.Adam(1e-3),#
                loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
                metrics=[tf.keras.metrics.AUC()])

    return model

In [182]:
def net_iter(model, ep=10, batch_size=64):
    history = model.fit(tr_data,
                        train_y_o,
                        batch_size=batch_size,
                        epochs=ep,
                        validation_data=(vd_data, valid_y_o),
                        shuffle=True,
                        verbose=0)    
    return history.history

In [183]:
summary_data = pd.DataFrame(data=[[0, 0, 0, 0, 0, 0, 0, 0]], 
                            columns=['layers', 'filters', 'neurons', 'mean_train_score', 'mean_val_score', 'variance', 'l_b', 'r_b'])
cnt = 0
f_n = [128, 256]
l_c = [1, 2]
nr = [32, 64]
epochs = 15

# net_res = []

for l_c_i in l_c:
    for f_n_i in f_n:
        for nr_i in nr:
            err_score = []
            err_val_score = []
            print(f"Модель: слоёв {l_c_i}, фильтров {f_n_i}, нейронов {nr_i}, эпох {epochs}\n")
            model = net_gen(filters=f_n_i, lay_cnt=l_c_i, neurons=nr_i)
            model.summary()
            for i in range(5):
                print(f"\tИтерация: {i+1}")
                hist = net_iter(model, ep=epochs)
                # net_res.append(hist)
                err_score.append(hist[list(hist.keys())[1]][-1])
                err_val_score.append(hist[list(hist.keys())[3]][-1])

            vr = np.var(err_val_score)
            l_b, r_b = calculate_confidence_interval(err_val_score)
            mean_err_score, mean_err_val_score = np.mean(err_score), np.mean(err_val_score)
            print(f"Ошибки на train. Средняя: {mean_err_score} список: {err_score}")
            print(f"Ошибки на valid. Средняя: {mean_err_val_score} список: {err_val_score}")
            print(f"Дисперсия ошибки на valid: {vr}")
            print(f"Доверительный интервал: {l_b} - {r_b}\n\n")
            summary_data.loc[cnt, ['layers', 'filters', 'neurons', 'mean_train_score', 
                                    'mean_val_score', 'variance', 'l_b', 'r_b']] = [l_c_i, f_n_i, 
                                    nr_i, np.mean(err_score), np.mean(err_val_score), vr, l_b, r_b]
            cnt += 1

Модель: слоёв 1, фильтров 128, нейронов 32, эпох 15

Model: "sequential_33"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_36 (Conv1D)           (None, 20, 128)           512       
_________________________________________________________________
dropout_59 (Dropout)         (None, 20, 128)           0         
_________________________________________________________________
flatten_32 (Flatten)         (None, 2560)              0         
_________________________________________________________________
dense_64 (Dense)             (None, 32)                81952     
_________________________________________________________________
dropout_60 (Dropout)         (None, 32)                0         
_________________________________________________________________
dense_65 (Dense)             (None, 5)                 165       
Total params: 82,629
Trainable params: 82,629
Non-trainable params

In [184]:
summary_data

,layers,filters,neurons,mean_train_score,mean_val_score,variance,l_b,r_b
0,1.0,128.0,32.0,0.816288,0.816312,0.0,0.816312,0.816312
1,1.0,128.0,64.0,0.816253,0.816312,0.0,0.816312,0.816312
2,1.0,256.0,32.0,0.816239,0.816312,0.0,0.816312,0.816312
3,1.0,256.0,64.0,0.816254,0.816312,0.0,0.816312,0.816312
4,2.0,128.0,32.0,0.816289,0.816312,0.0,0.816312,0.816312
5,2.0,128.0,64.0,0.816262,0.816312,0.0,0.816312,0.816312
6,2.0,256.0,32.0,0.816219,0.816312,0.0,0.816312,0.816312
7,2.0,256.0,64.0,0.816264,0.816312,0.0,0.816312,0.816312


In [185]:
summary_data.sort_values('mean_val_score', ascending=False)

,layers,filters,neurons,mean_train_score,mean_val_score,variance,l_b,r_b
0,1.0,128.0,32.0,0.816288,0.816312,0.0,0.816312,0.816312
1,1.0,128.0,64.0,0.816253,0.816312,0.0,0.816312,0.816312
2,1.0,256.0,32.0,0.816239,0.816312,0.0,0.816312,0.816312
3,1.0,256.0,64.0,0.816254,0.816312,0.0,0.816312,0.816312
4,2.0,128.0,32.0,0.816289,0.816312,0.0,0.816312,0.816312
5,2.0,128.0,64.0,0.816262,0.816312,0.0,0.816312,0.816312
6,2.0,256.0,32.0,0.816219,0.816312,0.0,0.816312,0.816312
7,2.0,256.0,64.0,0.816264,0.816312,0.0,0.816312,0.816312


In [186]:
# конфигурирование слоев нейросети
model = Sequential()

# слои нейросети отвественные за свертку и max-pooling
model.add(Conv1D(128, 3, padding='same', activation='sigmoid', input_shape=(tr_data.shape[1], 1))) # sigmoid'))relu
# model.add(Conv1D(128, 3, activation='relu'))
# model.add(MaxPooling1D(pool_size=2))
# model.add(Dropout(0.25))
# if lay_cnt == 2:
#     model.add(Conv1D(128, 3, padding='same', activation='relu'))
#     model.add(MaxPooling1D(pool_size=2))
#     model.add(Dropout(0.25))

# полносвязные слои нейронной сети
model.add(Flatten())
model.add(Dense(32, activation='sigmoid')) # 'relu'))
# model.add(Dense(neurons, activation='relu'))))
model.add(Dropout(0.5))
model.add(Dense(5, activation='softmax')) # sigmoid'))

# компиляция модели
model.compile(optimizer=tf.keras.optimizers.Adam(1e-3),#
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=[tf.keras.metrics.AUC()])

In [187]:
model.summary()

Model: "sequential_41"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_48 (Conv1D)           (None, 20, 128)           512       
_________________________________________________________________
flatten_40 (Flatten)         (None, 2560)              0         
_________________________________________________________________
dense_80 (Dense)             (None, 32)                81952     
_________________________________________________________________
dropout_79 (Dropout)         (None, 32)                0         
_________________________________________________________________
dense_81 (Dense)             (None, 5)                 165       
Total params: 82,629
Trainable params: 82,629
Non-trainable params: 0
_________________________________________________________________


In [188]:
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)

In [189]:
hist = model.fit(tr_data,
                 train_y_o,
                 batch_size=64,
                 epochs=1000,
                 validation_data=(vd_data, valid_y_o),
                 callbacks=[callback], 
                 shuffle=True,
                 verbose=1)

Epoch 1/1000
243/243 [==============================] - 2s 8ms/step - loss: 0.9937 - auc_13: 0.8633 - val_loss: 0.8675 - val_auc_13: 0.9057
Epoch 2/1000
243/243 [==============================] - 2s 7ms/step - loss: 0.9083 - auc_13: 0.8894 - val_loss: 0.8377 - val_auc_13: 0.9115
Epoch 3/1000
243/243 [==============================] - 2s 7ms/step - loss: 0.8768 - auc_13: 0.8982 - val_loss: 0.8344 - val_auc_13: 0.9108
Epoch 4/1000
243/243 [==============================] - 2s 7ms/step - loss: 0.8739 - auc_13: 0.8990 - val_loss: 0.8192 - val_auc_13: 0.9133
Epoch 5/1000
243/243 [==============================] - 2s 7ms/step - loss: 0.8638 - auc_13: 0.9018 - val_loss: 0.8122 - val_auc_13: 0.9139
Epoch 6/1000
243/243 [==============================] - 2s 6ms/step - loss: 0.8565 - auc_13: 0.9037 - val_loss: 0.8067 - val_auc_13: 0.9146
Epoch 7/1000
243/243 [==============================] - 2s 6ms/step - loss: 0.8422 - auc_13: 0.9067 - val_loss: 0.8021 - val_auc_13: 0.9156
Epoch 8/1000
243/243

## Задание 2

2. Предобучаем word2vec и его эмбединга инициализируем сетку, как влияет на качество?

array([5, 4, 5, ..., 4, 1, 5])